# 3 - API, Crawler, JSON e HTML

### Quais dados podemos obter sobre vereadoras e vereadores da cidade de São Paulo?

Links:
- [CMSP - Dados disponibilizados em formato aberto](http://www.saopaulo.sp.leg.br/transparencia/dados-abertos/dados-disponibilizados-em-formato-aberto/)
- [SPLEGIS - Sistema do Processo Legislativo](http://splegisws.camara.sp.gov.br/ws/ws2.asmx)
- [SPLEGIS - Lista de vereadores](http://splegisws.camara.sp.gov.br/ws/ws2.asmx?op=VereadoresCMSPJSON) 


#### Vamos lá!

In [ ]:
import requests
import pandas as pd

In [ ]:
url_api = 'http://splegisws.camara.sp.gov.br/ws/ws2.asmx/VereadoresCMSPJSON'

In [ ]:
resultados = requests.get(url_api)
resultados.status_code

In [ ]:
resultados.content

In [ ]:
type(resultados.content)

In [ ]:
resultados_json = resultados.json()
resultados_json

In [ ]:
type(resultados_json)

In [ ]:
len(resultados_json)

In [ ]:
resultados_json[0]

In [ ]:
type(resultados_json[0])

In [ ]:
lista_novos_dados = []

for item in resultados_json:
    novos_dados = {
        'nome': item.get('nome'),
        'qtd_mandatos': len(item.get('mandatos')),
        'qtd_cargos': len(item.get('cargos'))
    } 
    
    lista_novos_dados.append(novos_dados)

In [ ]:
resultados_df = pd.DataFrame(lista_novos_dados)
resultados_df.head(5)

In [ ]:
resultados_df.sort_values(['qtd_mandatos', 'qtd_cargos'], ascending=False).head(10)

### Podemos salvar o que extraímos em csv?

<img src="https://media.giphy.com/media/hVYVYZZBgF50k/giphy.gif" align="left">

In [ ]:
resultados_df.to_csv('lista_vereadores.csv', index=False)

#### E o que fazemos nesse caso?

In [ ]:
teste = requests.get('http://quotes.toscrape.com/tag/humor/')
teste.status_code

In [ ]:
teste.content

In [ ]:
teste.json()

## Brincando com o Scrapy

[Quotes to Scrape](http://quotes.toscrape.com/tag/humor/)

<img src="imagens/vereadoressp.png">

In [ ]:
%%writefile quotes.py
import scrapy


class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    
    def start_requests(self):
        urls = [
            'http://quotes.toscrape.com/tag/humor/page/1/',
            'http://quotes.toscrape.com/tag/humor/page/2/',
        ]
        
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.xpath('span/small/text()').get(),
            }

In [ ]:
!ls

In [ ]:
!scrapy runspider quotes.py -o quotes.json

In [ ]:
%%writefile items.py
import scrapy

class Vereador(scrapy.Item):
    
    nome = scrapy.Field()
    email = scrapy.Field()
    rede_social = scrapy.Field()

In [ ]:
%%writefile vereadores_sp.py
import scrapy
from scrapy.loader import ItemLoader
from items import Vereador

class VereadoresSpider(scrapy.Spider):
    name = 'veradores_sp'
    start_urls = ['http://www.saopaulo.sp.leg.br/vereadores/']

    def parse(self, response):      
        lista_veradores = response.css('p.vereador-name a::attr(href)').getall()
        
        for item in lista_veradores:
            yield response.follow(item, self.parse_vereador)
    
    
    def parse_vereador(self, response):
        
        item_loader = ItemLoader(Vereador(), response)
        
        item_loader.add_css('nome', 'p.vereador-name a::text')
        item_loader.add_css('email', 'div.vereador-data ul li a::text')
        item_loader.add_xpath('rede_social', '/html/body/div[2]/div/div[3]/div/div/article/header/div[2]/ul/li[5]/a/@href')

    
        return item_loader.load_item()
    
    

In [ ]:
!ls

In [ ]:
!scrapy runspider vereadores_sp.py -o vereadores.json

In [ ]:
!ls

#### Ah massa! Fizemos tudo. Temos um forma automatizada de extrair, analisar e salvar esse dado... O que acontece agora?

# A estrutura HTML vai mudar!

<img src="https://media.tenor.com/images/75457a53b6ad99b2c857353eb4e21f76/tenor.gif" align="left">